In [ ]:
# ============================================================================
# CELDA 1: IMPORTACIÓN DE LIBRERÍAS Y CONFIGURACIÓN INICIAL
# ============================================================================
%run ./00_template.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from pathlib import Path

# Librerías de Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# Configuración de gráficos
sns.set_style("whitegrid")
print("✅ Librerías cargadas y entorno configurado.")

In [ ]:
# ============================================================================
# CELDA 2: CARGA, TRANSFORMACIÓN Y LIMPIEZA DE DATOS
# ============================================================================
# 1. Definir rutas
INPUT_CSV = OUTPUTS_DIR / "accesibilidad_otp_final.csv"
OUTPUT_PARQUET = OUTPUTS_DIR / "comunas_accesibilidad_wide.parquet"

print(f"📂 Leyendo archivo generado por OTP: {INPUT_CSV}")

if not INPUT_CSV.exists():
    raise FileNotFoundError("❌ No existe el CSV. Ejecuta el Notebook 03 primero.")

# 2. Cargar datos brutos
df_long = pd.read_csv(INPUT_CSV)

# 3. Pivotear (Transformar de formato Largo a Ancho)
# Fila = Comuna, Columnas = Categorías de servicio, Valor = Minutos
acc = df_long.pivot_table(
    index="cod", 
    columns="cat", 
    values="minutos", 
    aggfunc="min" 
).reset_index()

# Renombrar para estandarizar
acc = acc.rename(columns={"cod": "cod_comuna"})

# 4. Enriquecer con nombres de comunas (desde el GeoPackage)
gdf_base = gpd.read_file(RUTA_GPKG, layer="comunas_rm_censo")
mapping = gdf_base[["CUT_COM", "COMUNA"]].copy()
mapping["CUT_COM"] = mapping["CUT_COM"].astype(str)
acc["cod_comuna"] = acc["cod_comuna"].astype(str)

acc = acc.merge(mapping, left_on="cod_comuna", right_on="CUT_COM", how="left")
acc = acc.rename(columns={"NOM_COM": "comuna"})
acc = acc.drop(columns=["CUT_COM"])

# 5. Imputación de Valores Nulos
# Si un valor es NaN, significa que no hay ruta o está muy lejos.
# Lo reemplazamos con un valor alto (ej: 120 min) para "penalizar" esa comuna en el clustering.
numeric_cols = [c for c in acc.columns if c not in ["cod_comuna", "comuna"]]

for col in numeric_cols:
    nulls = acc[col].isna().sum()
    if nulls > 0:
        val_penalizacion = 120 # 2 horas
        print(f"⚠️ {col}: {nulls} comunas sin cobertura. Imputando {val_penalizacion} min.")
        acc[col] = acc[col].fillna(val_penalizacion)

# 6. Guardar dataset listo para ML
acc.to_parquet(OUTPUT_PARQUET, index=False)

print(f"\n✅ Datos procesados y guardados en: {OUTPUT_PARQUET}")
print("   Dimensiones:", acc.shape)
acc.head()

In [ ]:
# ============================================================================
# CELDA 3: ESTADÍSTICAS DESCRIPTIVAS
# ============================================================================
print("📊 Resumen estadístico de tiempos de viaje (minutos):")
desc = acc[numeric_cols].describe().T
display(desc)

# Validaciones de seguridad
assert len(acc) >= 5, f"❌ Muy pocas comunas ({len(acc)}). Revisa el Notebook 03."
assert len(numeric_cols) >= 2, "❌ Necesitas al menos 2 categorías de servicios para hacer clustering."

In [ ]:
# ============================================================================
# CELDA 4: PREPROCESAMIENTO Y PCA (CORREGIDA)
# ============================================================================
# 1. FILTRADO INTELIGENTE DE COLUMNAS
# Pandas tiene una función para seleccionar solo números automáticamente.
# Esto evita que se cuelen columnas de texto como "Santiago".
X = acc.select_dtypes(include=['number']).copy()

# Por seguridad, eliminamos 'cod_comuna' si es que quedó como número por error
if "cod_comuna" in X.columns:
    X = X.drop(columns=["cod_comuna"])

# Verificar qué columnas vamos a usar
print("✅ Columnas seleccionadas para el análisis (Features):")
print(X.columns.tolist())

# Asegurarnos de que no esté vacío
if X.empty or len(X.columns) < 2:
    raise ValueError("❌ Error: No se encontraron columnas numéricas suficientes.")

# 2. Estandarizar (StandardScaler)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. PCA para visualizar en 2D
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print(f"\n📈 Varianza explicada: {pca.explained_variance_ratio_.sum():.2%}")

# 4. Graficar
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], alpha=0.8, s=100, color='royalblue')

# Etiquetar algunas comunas para referencia
# Usamos acc["comuna"] (o la columna de nombre que tengas) para las etiquetas
nombres_comunas = acc["comuna"] if "comuna" in acc.columns else acc["COMUNA"]

for i, txt in enumerate(nombres_comunas):
    # Solo etiquetamos si están lejos del centro (outliers visuales) para no llenar el mapa
    if abs(X_pca[i,0]) > 2 or abs(X_pca[i,1]) > 2:
        plt.annotate(txt, (X_pca[i,0]+0.1, X_pca[i,1]), fontsize=9)

plt.title("Mapa de Similitud de Comunas (PCA)")
plt.xlabel("Componente Principal 1 (Accesibilidad General)")
plt.ylabel("Componente Principal 2 (Perfil de Servicios)")
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()

In [ ]:
# ============================================================================
# CELDA 5: DETERMINACIÓN DEL NÚMERO ÓPTIMO DE CLUSTERS (K)
# ============================================================================
scores = []
K_range = range(2, 8) # Probaremos dividir Santiago en 2 a 7 zonas

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    scores.append(score)

plt.figure(figsize=(8, 4))
plt.plot(K_range, scores, marker="o", linestyle="-", color="teal", linewidth=2)
plt.xlabel("Número de Clusters (k)")
plt.ylabel("Silhouette Score (Mayor es mejor)")
plt.title("Calidad del Agrupamiento")
plt.xticks(K_range)
plt.grid(True)
plt.show()

In [ ]:
# ============================================================================
# CELDA 6: APLICACIÓN DEL MODELO FINAL (K-MEANS)
# ============================================================================
# --- AJUSTA ESTE VALOR SEGÚN EL GRÁFICO ANTERIOR ---
K_OPTIMO = 4
# ---------------------------------------------------

kmeans = KMeans(n_clusters=K_OPTIMO, random_state=42, n_init=10)
acc["cluster"] = kmeans.fit_predict(X_scaled)

print(f"✅ Clustering completado con K={K_OPTIMO}")
print("\nDistribución de comunas por cluster:")
print(acc["cluster"].value_counts().sort_index())

# Mostrar ejemplos por cluster
for k in range(K_OPTIMO):
    print(f"\n📁 Cluster {k}:", acc[acc["cluster"]==k]["COMUNA"].head(5).tolist(), "...")

In [ ]:
# ============================================================================
# CELDA 7: INTERPRETACIÓN DE LOS PERFILES (CORREGIDA)
# ============================================================================
# 1. Identificar dinámicamente las columnas numéricas de servicios
# Buscamos solo columnas numéricas
cols_analisis = acc.select_dtypes(include=['number']).columns.tolist()

# Limpiamos la lista: No queremos promediar el 'cluster' ni códigos
columnas_a_excluir = ["cluster", "cod_comuna", "cod", "CUT_COM"]
cols_analisis = [c for c in cols_analisis if c not in columnas_a_excluir]

print(f"📊 Generando perfil basado en {len(cols_analisis)} servicios:")
print(cols_analisis)

# 2. Calcular el tiempo promedio por cluster
# Agrupamos por cluster y solo pedimos el promedio de las columnas filtradas
perfil = acc.groupby("cluster")[cols_analisis].mean()

print("\n⏱️ Tiempos promedio de viaje (minutos) por cluster:")
print("(Verde 🟩 = Menor tiempo = Mejor accesibilidad)")
print("(Rojo 🟥 = Mayor tiempo = Peor accesibilidad)")

# 3. Mostrar tabla con colores
# Usamos 'RdYlGn_r' invertido: Verde para valores bajos (bueno), Rojo para altos (malo)
display(perfil.style.background_gradient(cmap="RdYlGn_r"))

In [ ]:
# ============================================================================
# CELDA 8: VISUALIZACIÓN ESPACIAL (MAPA)
# ============================================================================
# 1. Cargar geometrías
gdf_map = gpd.read_file(RUTA_GPKG, layer="comunas_rm_censo")
gdf_map["CUT_COM"] = gdf_map["CUT_COM"].astype(str)

# 2. Unir resultados del clustering
gdf_final = gdf_map.merge(acc[["cod_comuna", "cluster"]], left_on="CUT_COM", right_on="cod_comuna", how="left")

# 3. Plotear
fig, ax = plt.subplots(figsize=(12, 12))

gdf_final.plot(
    column="cluster",
    cmap="viridis",        # Paleta de colores
    categorical=True,      # Tratar clusters como categorías, no números continuos
    legend=True,
    legend_kwds={'title': 'Grupo de Accesibilidad', 'loc': 'upper left'},
    missing_kwds={'color': 'lightgrey', 'label': 'Sin datos'},
    edgecolor="white",
    linewidth=0.5,
    ax=ax
)

ax.set_title(f"Zonas de Accesibilidad en Santiago (K={K_OPTIMO})", fontsize=16)
ax.set_axis_off()
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# CELDA 9: GUARDADO FINAL Y EXPORTACIÓN
# ============================================================================
# 1. Asignamos nombres legibles a los clusters (OPCIONAL, PERO RECOMENDADO)
# Ajusta estos nombres según lo que viste en tu mapa K=4 y la tabla de promedios.
# Ejemplo basado en tu imagen (verifica con tus datos reales cuál es cuál):
# Si Cluster 0 es lento (Maipú), Cluster 1 es medio (Centro), etc.

# Para saber cuál es cuál, miramos los promedios de nuevo:
print("Promedios para identificar etiquetas:")
display(acc.groupby("cluster")[cols_analisis].mean().mean(axis=1))

# Supongamos (¡Revisa los números!) que:
# 0 = Periferia Poniente
# 1 = Periferia Norte
# 2 = Cono Oriente (Tiempos más bajos)
# 3 = Eje Centro-Sur
# dic_etiquetas = {0: "Poniente", 1: "Norte", 2: "Oriente (Alta Acceso)", 3: "Centro-Sur"}
# acc["nombre_cluster"] = acc["cluster"].map(dic_etiquetas)

# 2. Guardar CSV Final
OUTPUT_FINAL = OUTPUTS_DIR / "comunas_accesibilidad_clusters.csv"
acc.to_csv(OUTPUT_FINAL, index=False)

print(f"💾 Archivo final guardado en: {OUTPUT_FINAL}")
print("¡Felicidades! Has completado el análisis de accesibilidad.")